In [25]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('covid_toy.csv')

In [3]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [20]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,0:5], df.iloc[:,5],test_size =0.2)

# here i am doing without using Column transformer

In [13]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [18]:
si = SimpleImputer() # here i am filling out the blank space of fever
X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.fit_transform(X_test[['fever']])

In [19]:
X_train_fever.shape

(80, 1)

In [24]:
oe = OrdinalEncoder(categories = [['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.fit_transform(X_test[['cough']])
X_train_cough.shape

(80, 1)

In [26]:
ohe = OneHotEncoder(drop = 'first',sparse_output= False, dtype = np.int32)

In [29]:
X_train_scaled = ohe.fit_transform(X_train[['gender', 'city']])
X_test_scaled = ohe.fit_transform(X_test[['gender', 'city']])
X_train_scaled.shape

(80, 4)

In [34]:
X_train_age = X_train.drop(columns= ['gender', 'fever', 'cough', 'city'])

In [36]:
X_train_final = np.concatenate((X_train_age,X_train_cough,X_train_scaled,X_train_fever), axis = 1)

In [42]:
X_train_final

array([[ 27.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        , 100.        ],
       [ 82.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        , 102.        ],
       [  8.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        , 101.        ],
       [ 20.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        , 100.86111111],
       [ 49.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        , 102.        ],
       [ 49.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  99.        ],
       [ 14.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  99.        ],
       [ 15.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        , 101.        ],
       [ 25.        ,   1.        ,   0.        ,   0.        ,
          1.    

# Now with ColumnTransfer

In [39]:
from sklearn.compose import ColumnTransformer

In [50]:
Ct = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(),['fever']),
    ('tnf2', OrdinalEncoder(categories = [['Mild', 'Strong']]),['cough']),
    ('tnf3', OneHotEncoder(drop='first',sparse_output=False, dtype = np.int32),['city','gender'])
],remainder = 'passthrough') 

In [51]:
X_train_Col = Ct.fit_transform(X_train)
X_test_Col = Ct.fit_transform(X_test)

In [53]:
X_train_Col

array([[100.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   1.        ,  27.        ],
       [102.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  82.        ],
       [101.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,   8.        ],
       [100.86111111,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  20.        ],
       [102.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  49.        ],
       [ 99.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  49.        ],
       [ 99.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  14.        ],
       [101.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   1.        ,  15.        ],
       [ 99.        ,   1.        ,   0.        ,   1.        ,
          0.    

In [46]:
X_train

,age,gender,fever,cough,city
93,27,Male,100.0,Mild,Kolkata
92,82,Female,102.0,Strong,Kolkata
77,8,Female,101.0,Mild,Kolkata
7,20,Female,NaN,Strong,Mumbai
40,49,Female,102.0,Mild,Delhi
...,...,...,...,...,...
6,14,Male,101.0,Strong,Bangalore
98,5,Female,98.0,Strong,Mumbai
90,59,Female,99.0,Strong,Delhi
60,24,Female,102.0,Strong,Bangalore
